# Which NYC Subway Stations Are at Highest Risk of Platform Overflow?

This analysis uses MTA hourly ridership data for 2025 (with rolling updates from 2026) combined with Additional Platform Time (APT) metrics to identify stations where passenger volume and service stress converge. APT data is published per subway line, not per station; the dashboard bridges this gap using a station-to-route mapping that computes a weighted-average APT for each station complex based on the lines that serve it. 

The Capacity Stress Index (CSI) synthesizes peak-hour concentration and raw volume into a single risk score, calibrated against the full distribution of 2025 ridership — a post-stabilization year that serves as the cleanest available baseline for what "normal" NYC subway load looks like.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.dates import MonthLocator, DateFormatter

import seaborn as sns
import plotly.express as px
from pathlib import Path

PROCESSED = Path("../data/processed")

In [ ]:
df = pd.read_parquet(PROCESSED / "ridership_transformed")
df["year"] = df["year"].astype(int)
df["month"] = df["month"].astype(int)
df.head()

In [ ]:
df.tail()

In [ ]:
csi_df = pd.read_parquet(PROCESSED / "csi")
csi_df["year"] = csi_df["year"].astype(int)
csi_df["month"] = csi_df["month"].astype(int)
csi_df.head()

In [ ]:
csi_df["danger_zone"].unique()

In [ ]:
csi_df[csi_df["danger_zone"] == 1][["station_complex", "year", "month", "csi", "apt_minutes"]].sort_values("csi",ascending=False).head(10)

In [ ]:
csi_df[csi_df["danger_zone"] == 1][["station_complex", "year", "month", "csi", "apt_minutes"]].sort_values("apt_minutes",ascending=False).head(10)

In [ ]:
print(csi_df["volume_component"].min(), csi_df["volume_component"].max())
print(csi_df["hhi_monthly_concentration"].min(), csi_df["hhi_monthly_concentration"].max())
print(csi_df["csi"].min(), csi_df["csi"].max())


In [ ]:
print(csi_df["apt_minutes"].dropna().min(), csi_df["apt_minutes"].dropna().max())

In [ ]:
borough_totals = df.groupby("borough")["ridership"].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8, 4))
borough_totals.plot(kind="bar", ax=ax, color=sns.color_palette("muted"))
ax.set_title("Total Ridership by Borough — 2025")
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f"{x/1e6:.0f}M"))
ax.set_xlabel(""); ax.set_ylabel("Riders")
plt.tight_layout(); plt.show()

In [ ]:
month_names = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

## Average Hourly Ridership per Day of Week

For a specific station and month in a year.

In [ ]:
subset = csi_df.nlargest(1, "csi")[["station_complex", "year", "month", "csi"]]
top_5 = [tuple(row) for row in subset.itertuples(index=False)]

for station, yr, m, csi in top_5:
    pivot = df[df["station_complex"] == station] \
        .groupby(["day_of_week", "hour_of_day"])["ridership"].mean().unstack()
    fig, ax = plt.subplots(figsize=(14, 3))
    sns.heatmap(pivot, ax=ax, cmap="YlOrRd", linewidths=0.3,
                xticklabels=[f"{h}:00" for h in range(24)],
                yticklabels=["Sun","Mon","Tue","Wed","Thu","Fri","Sat"])
    ax.set_title(f"{station}, {month_names[m]} {yr}, CSI={csi:.3f}")
    plt.tight_layout(); plt.show()

In [ ]:
subset = csi_df.iloc[csi_df.groupby("station_complex")["csi"].idxmax()].nlargest(5, "csi")[["station_complex", "year", "month", "csi"]]
top_5 = [tuple(row) for row in subset.itertuples(index=False)]

for station, yr, m, csi in top_5:
    pivot = df[df["station_complex"] == station] \
        .groupby(["day_of_week", "hour_of_day"])["ridership"].mean().unstack()
    fig, ax = plt.subplots(figsize=(14, 3))
    sns.heatmap(pivot, ax=ax, cmap="YlOrRd", linewidths=0.3,
                xticklabels=[f"{h}:00" for h in range(24)],
                yticklabels=["Sun","Mon","Tue","Wed","Thu","Fri","Sat"])
    ax.set_title(f"{station}, {month_names[m]} {yr}, CSI={csi:.3f}")
    plt.tight_layout(); plt.show()

## Latest Month Analysis

In [ ]:
latest_year = csi_df["year"].max()
latest_month = csi_df[csi_df["year"] == latest_year]["month"].max()

print(month_names[latest_month], latest_year)

In [ ]:
csi_latest = csi_df[(csi_df["year"] == latest_year) & (csi_df["month"] == latest_month)]

# Pull monthly_ridership from df (already computed), deduplicate to one row per station/month
df_monthly = (
    df[(df["year"] == latest_year) & (df["month"] == latest_month)]
    [["station_complex", "year", "month", "monthly_ridership"]]
    .drop_duplicates(subset=["station_complex", "year", "month"])
)

monthly_latest = csi_latest.merge(df_monthly, on=["station_complex", "year", "month"], how="left")

top10 = monthly_latest.nlargest(10, "monthly_ridership")[
    ["station_complex", "borough", "monthly_ridership", "csi",
     "apt_minutes", "num_lines_serving"]
]

top10.style \
    .background_gradient(subset=["apt_minutes"], cmap="Blues") \
    .background_gradient(subset=["csi"], cmap="Reds") \
    .format({
        "monthly_ridership": "{:,.0f}",
        "csi": "{:.3f}",
        "apt_minutes": "{:.1f}",
        "num_lines_serving": "{:.0f}",
    })

In [ ]:
danger_latest = csi_df[(csi_df["year"] == latest_year) & (csi_df["month"] == latest_month) & (csi_df["danger_zone"] == 1)]

# Pull monthly_ridership from df (already computed), deduplicate to one row per station/month
df_monthly = (
    df[(df["year"] == latest_year) & (df["month"] == latest_month)]
    [["station_complex", "year", "month", "monthly_ridership"]]
    .drop_duplicates(subset=["station_complex", "year", "month"])
)

danger_monthly_latest = danger_latest.merge(df_monthly, on=["station_complex", "year", "month"], how="left")

danger_top20 = danger_monthly_latest.nlargest(20, "monthly_ridership")[
    ["station_complex", "borough", "monthly_ridership", "csi",
     "apt_minutes", "num_lines_serving"]
]

danger_top20.style \
    .background_gradient(subset=["apt_minutes"], cmap="Blues") \
    .background_gradient(subset=["csi"], cmap="Reds") \
    .format({
        "monthly_ridership": "{:,.0f}",
        "csi": "{:.3f}",
        "apt_minutes": "{:.1f}",
        "num_lines_serving": "{:.0f}",
    })

In [ ]:
fig = px.scatter(
    monthly_latest.dropna(subset=["apt_minutes"]),
    x="hhi_monthly_concentration",
    y="apt_minutes",
    size="monthly_ridership",
    color="borough",
    hover_name="station_complex",
    hover_data={"num_lines_serving": True, "csi": ":.3f"},
    # text=monthly_latest.dropna(subset=["apt_minutes"])
    #     .nlargest(10, "csi")["station_complex"],
    title=f"Capacity Stress Index: Peak Concentration vs. Additional Platform Wait Time (2025 Baseline), {month_names[latest_month]} {latest_year}",
    labels={
        "hhi_monthly_concentration": "Ridership Concentration (Herfindahl Index)",
        "apt_minutes": "Weighted Avg Additional Platform Time (min)",
    },
    size_max=50,
)
fig.update_traces(textposition="top center")
fig.show()

In [ ]:
fig = px.scatter(
    monthly_latest.dropna(subset=["apt_minutes"]),
    x="hhi_monthly_concentration",
    y="apt_minutes",
    size="monthly_ridership",
    color="danger_zone",
    hover_name="station_complex",
    hover_data={"num_lines_serving": True, "csi": ":.3f"},
    # text=monthly_latest.dropna(subset=["apt_minutes"])
    #     .nlargest(10, "csi")["station_complex"],
    title=f"Capacity Stress Index: Peak Concentration vs. Additional Platform Wait Time (2025 Baseline), {month_names[latest_month]} {latest_year}",
    labels={
        "hhi_monthly_concentration": "Ridership Concentration (Herfindahl Index)",
        "apt_minutes": "Weighted Avg Additional Platform Time (min)",
    },
    size_max=50,
)
fig.update_traces(textposition="top center")
fig.show()

In [ ]:
fig = px.scatter(
    monthly_latest.dropna(subset=["apt_minutes"]),
    x="hhi_monthly_concentration",
    y="apt_minutes",
    size="volume_component",
    color="borough",
    hover_name="station_complex",
    hover_data={"num_lines_serving": True, "csi": ":.3f", "volume_component": ":.3f", "hhi_monthly_concentration": ":.3f", "apt_minutes": ":.3f"},
    # text=monthly_latest.dropna(subset=["apt_minutes"])
    #     .nlargest(10, "csi")["station_complex"],
    title=f"Capacity Stress Index: Peak Concentration vs. Weighted Platform Wait Time (2025 Baseline), {month_names[latest_month]} {latest_year}",
    labels={
        "hhi_monthly_concentration": "Ridership Concentration (HHI)",
        "apt_minutes": "Weighted Avg Additional Platform Time (min)",
    },
    # size_max=50,
)
fig.update_traces(textposition="top center")
fig.show()

In [ ]:
fig = px.scatter(
    monthly_latest.dropna(subset=["apt_minutes"]),
    x="hhi_monthly_concentration",
    y="apt_minutes",
    size="volume_component",
    color="danger_zone",
    hover_name="station_complex",
    hover_data={"num_lines_serving": True, "csi": ":.3f", "volume_component": ":.3f", "hhi_monthly_concentration": ":.3f", "apt_minutes": ":.3f"},
    # text=monthly_latest.dropna(subset=["apt_minutes"])
    #     .nlargest(10, "csi")["station_complex"],
    title=f"Capacity Stress Index: Peak Concentration vs. Weighted Platform Wait Time (2025 Baseline), {month_names[latest_month]} {latest_year}",
    labels={
        "hhi_monthly_concentration": "Ridership Concentration (HHI)",
        "apt_minutes": "Weighted Avg Additional Platform Time (min)",
    },
    # size_max=50,
)
fig.update_traces(textposition="top center")
fig.show()

In [ ]:
fig = px.scatter(
    monthly_latest.dropna(subset=["apt_minutes"]),
    x="apt_minutes",
    y="hhi_monthly_concentration",
    size="volume_component",
    color="danger_zone",
    hover_name="station_complex",
    hover_data={"num_lines_serving": True, "csi": ":.3f", "volume_component": ":.3f", "hhi_monthly_concentration": ":.3f", "apt_minutes": ":.3f"},
    # text=monthly_latest.dropna(subset=["apt_minutes"])
    #     .nlargest(10, "csi")["station_complex"],
    title=f"Capacity Stress Index: Peak Concentration vs. Weighted Platform Wait Time (2025 Baseline), {month_names[latest_month]} {latest_year}",
    labels={
        "hhi_monthly_concentration": "Ridership Concentration (HHI)",
        "apt_minutes": "Weighted Avg Additional Platform Time (min)",
    },
    # size_max=50,
)
fig.update_traces(textposition="top center")
fig.show()

In [ ]:
fig = px.scatter(
    monthly_latest.dropna(subset=["apt_minutes"]),
    x="apt_minutes",
    y="csi",
    size="volume_component",
    color="danger_zone",
    hover_name="station_complex",
    hover_data={"num_lines_serving": True, "csi": ":.3f", "volume_component": ":.3f", "hhi_monthly_concentration": ":.3f", "apt_minutes": ":.3f"},
    # text=monthly_latest.dropna(subset=["apt_minutes"])
    #     .nlargest(10, "csi")["station_complex"],
    title=f"Capacity Stress Index: Peak Concentration vs. Weighted Platform Wait Time (2025 Baseline), {month_names[latest_month]} {latest_year}",
    labels={
        "csi": "Capacity Stress Index (CSI)",
        "hhi_monthly_concentration": "Ridership Concentration (HHI)",
        "apt_minutes": "Weighted Avg Additional Platform Time (min)",
    },
    # size_max=50,
)
fig.update_traces(textposition="top center")
fig.show()

## Capacity Stress Index Trends

In [ ]:
high_csi = csi_df.iloc[csi_df[(csi_df["year"]==2025) & (csi_df["month"]>=6)].groupby("station_complex")["csi_6mo_rolling"].idxmax()].nlargest(5, "csi_6mo_rolling")["station_complex"].tolist()
trend = csi_df[csi_df["station_complex"].isin(high_csi)].copy()
trend["date"] = pd.to_datetime(trend[["year","month"]].assign(day=1))
trend = trend.sort_values(["year", "month"])
trend = trend[trend["date"] >= "2025-06-01"]
trend.head()

In [ ]:

fig, ax = plt.subplots(figsize=(12, 5))
for station, grp in trend.groupby("station_complex"):
    ax.plot(grp["date"], grp["csi_6mo_rolling"], label=station, marker='o', markersize=4)

ax.set_title("6-Month Rolling CSI — High-CSI Stations")
ax.set_ylabel("CSI (rolling avg)")

ax.xaxis.set_major_locator(MonthLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y-%m"))
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)

plt.show()

In [ ]:
high_csi = csi_df.iloc[csi_df[(csi_df["year"]==2025) & (csi_df["month"]>=6) & (csi_df["danger_zone"] == 1)].groupby("station_complex")["csi_6mo_rolling"].idxmax()].nlargest(5, "csi_6mo_rolling")["station_complex"].tolist()
trend = csi_df[csi_df["station_complex"].isin(high_csi)].copy()
trend["date"] = pd.to_datetime(trend[["year","month"]].assign(day=1))
trend = trend.sort_values(["year", "month"])
trend = trend[trend["date"] >= "2025-06-01"]
trend.head()

In [ ]:

fig, ax = plt.subplots(figsize=(12, 5))
for station, grp in trend.groupby("station_complex"):
    ax.plot(grp["date"], grp["csi_6mo_rolling"], label=station, marker='o', markersize=4)

ax.set_title("6-Month Rolling CSI — High-Risk Stations")
ax.set_ylabel("CSI (rolling avg)")

ax.xaxis.set_major_locator(MonthLocator())
ax.xaxis.set_major_formatter(DateFormatter("%Y-%m"))
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2)

plt.show()